# 1 - Data Reading

In [43]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#read from csv
test_df = pd.read_csv("sat-test-data.csv.dat", header=None)
train_df = pd.read_csv("sat-train.csv.dat", header =None)


# 2 - Data Pre-Processing

## 2.1 - Missing Values

In [45]:
import  scipy.stats as stats
import numpy.ma as ma

######replace by zero - black colour in RGB
#train_df = np.nan_to_num(train_df)
#test_df = np.nan_to_num(test_df)

######replace by 255 - white colour in RGB
#train_df = train_df.fillna(255)
#test_df = train_df.fillna(255)

######column minimum
#train_df = train_df.fillna(train_df.min())
#test_df = test_df.fillna(test_df.min())

######column maximum
#train_df = train_df.fillna(train_df.max())
#test_df = test_df.fillna(test_df.max())


#column mean
#train_df = train_df.fillna(train_df.mean())
#test_df = test_df.fillna(test_df.mean())

#column median 
#train_df = train_df.fillna(train_df.median())
#test_df = test_df.fillna(test_df.median())

# in the analysis of the rows we have to take into account that each four consecutive rows describe a pixel.
# Each of these four rows stands for: Green, R, IR, IR. These values refer to different things, and are analysed independently because of that.

Y_train = train_df[len(train_df.columns)-1]
X_train = train_df.drop(len(train_df.columns)-1,axis=1)

num_row = X_train[0].count()
num_column = len(X_train.columns)

print(num_row)
print(num_column)

#make four datasets

#than you can apply the row operations to replace the missing values by row

#example with for cycle
for x in range(0, num_row):
    row_index = [x]
    column_index = [x]
    print(test_df.iloc[row_index,column_index])
    
    
#the four datasets can be used to make the distributions




#row spectral mean

#row spectral median

#row spectral minimum

#row spectral maximum

# distribution of the spectral values. Get random value from this sample


3504
36
      0
0  80.0
       1
1  102.0
       2
2  106.0
      3
3  76.0
      4
4  76.0
    5
5 NaN
    6
6 NaN
      7
7  76.0
      8
8  72.0
    9
9 NaN
      10
10  90.0
      11
11  72.0
    12
12 NaN
    13
13 NaN
    14
14 NaN
    15
15 NaN
      16
16  88.0
       17
17  107.0
       18
18  113.0
      19
19  85.0
    20
20 NaN
       21
21  112.0
       22
22  113.0
      23
23  92.0
      24
24  93.0
    25
25 NaN
       26
26  100.0
      27
27  79.0
      28
28  75.0
      29
29  79.0
      30
30  81.0
    31
31 NaN
      32
32  90.0
       33
33  100.0
      34
34  92.0
      35
35  70.0


IndexError: positional indexers are out-of-bounds

# 3 - Choose Model

Nota: adicionar neural networks - new in v 0.18

First goal: discover which type of analyses works better
Second Goal: tune the model

In [35]:
#inspired in http://machinelearningmastery.com/compare-machine-learning-algorithms-python-scikit-learn/

import pandas
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# prepare data

Y_train = train_df[:,-1]
X_train = train_df[:,:-1]

#Y_train = train_df[len(train_df.columns)-1]
#X_train = train_df.drop(len(train_df.columns)-1,axis=1)

# prepare configuration for cross validation test harness
num_folds = 10
num_instances = len(X_train)
seed = 7

# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('NN', MLPClassifier(alpha=1))) 
models.append(('RF', RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)))
models.append(('AB', AdaBoostClassifier()))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy' # try with 'roc_auc', f1'

for name, model in models:
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

TypeError: unhashable type: 'slice'

# Submission in Kaggle

In [7]:
# Logistic Regression

adaboost = AdaBoostClassifier()

adaboost.fit(X_train, Y_train)

Y_pred = adaboost.predict(test_df)

adaboost.score(X_train, Y_train)

print(np.arange(Y_pred.size))

Y_pred = Y_pred.astype(int)
#np.hstack((Y_pred, np.vstack(np.arange(Y_pred.size))))

print(Y_pred)
np.savetxt("Pauliguel.csv", Y_pred, delimiter=",", fmt='%u')

[   0    1    2 ..., 1997 1998 1999]
[0 0 0 ..., 0 0 0]


# Pipeline

- Poderíamos tentar fazer alguma coisa a ver com anomaly, porque até agora é uma análise muito geral:
-- Pedir frequências das duas classes.
-- Pedir um gráfico com as distribuições dos valores para cada espectro.
-- Ver resultado quando se lida com os Missing values pela estratégia da distribuição
-- ter o cuidado de ver se os resultados não estão a ser influenciados porque o nosso algoritmo diz sempre a mesma classe: ver precision e recall

In [34]:
#ReadData
#XTrain, YTrain, rawTestData = importDataFromFiles()

#handle missing values

#for each different strategy for the missing values check which is the best model in the, get values for accuracy, auc, F1
#check both on the K fold, and on the test dataset

#pick the best strategy for missing values and  the best model and submit to kaggle 
